In [ ]:
import websocket
import json
from urllib.parse import quote

In [ ]:
ws = websocket.WebSocket()
website_id='google.com'
ws.connect(f'wss://plankton-app-5rfza.ondigitalocean.app/websites?scraping={website_id}')
ws.recv()

In [ ]:
ws.close()

In [ ]:
ws = websocket.WebSocket()
page_id = 'https://betway.com/es/sports/evt/10682299","websiteId'
website_id='betway.com'
# uri = 'wss://plankton-app-5rfza.ondigitalocean.app'
uri='ws://localhost:8000'
ws.connect(f'{uri}/games/{website_id}?game_id={page_id}')
ws.recv()

In [79]:
from core.models import Game


g = Game(id='https://pe.betano.com/cuotas-de-partido/fc-sheriff-tiraspol-fk-partizan-belgrade/29474553/', websiteId='web', )
g.dict(exclude_unset=True)

{'id': 'https://pe.betano.com/cuotas-de-partido/fc-sheriff-tiraspol-fk-partizan-belgrade/29474553/',
 'websiteId': 'web'}

In [42]:
from core.models import r
from redis.commands.search.query import Query, NumericFilter
from  redis.commands.search import aggregation
from redis.commands.search import reducers
from datetime import datetime, timezone

In [82]:
r.

In [ ]:
from core.webdriver import init_driver
from selenium.webdriver.common.by import By
from core.models import Bet
from time import sleep
driver = init_driver()
url='https://betway.com/es/sports/evt/10682245'
driver.get(url)

In [ ]:
sleep(10)
status_code = driver.find_element(By.CSS_SELECTOR, 'meta[http-equiv="status"]').get_attribute('content')
print(status_code == '404')

In [ ]:
def get_bids(page_url):
    for elm in driver.find_elements(By.CSS_SELECTOR, '.collapsablePanel'):
        collapsed_elm = elm.find_element(By.CSS_SELECTOR, '.collapsableContent')
        if 'empty' in collapsed_elm.get_attribute('class'):
            try:
                elm.click()
            except:
                pass
        try:
            header_elm = elm.find_element(By.CSS_SELECTOR, '.titleText>.title')
            cashout_elm = elm.find_element(By.CSS_SELECTOR, '.cashOutMarketIndicatorContainer')
        except:
            continue    
        try:
            labels_elms = elm.find_elements(By.CSS_SELECTOR, '.outcomeHeader, .outcomeItemHeader')
            odds_elms = elm.find_elements(By.CSS_SELECTOR, '.oddsDisplay')
        except:
            continue
        
        for labels_elm, odds_elm in zip(labels_elms, odds_elms):
            try:
                group = header_elm.text.strip()
                name = labels_elm.text.strip()
                bet = Bet(
                    id=f'{group}-{name}'.replace(' ', '-').lower(),
                    websiteId='betway.com',
                    gameId=page_url,
                    group=group, 
                    name=name, 
                    bet=float(odds_elm.text.replace(',', '.'))
                )
            except Exception as e:
                print('Error', e)

In [ ]:
get_bids(url)